#  ADM HW3 Group 25

### Libraries

In [14]:
import csv
import pandas as pd
import numpy as np
import nltk 
import functions
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk import word_tokenize
import string
from nltk.tokenize import RegexpTokenizer
#download the stop words
nltk.download("stopwords")
import json
import math

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\elena\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [35]:
from importlib import reload
reload(functions)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\elena\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


<module 'functions' from 'C:\\Users\\elena\\Documents\\DataScience\\ADM_18\\HW_3\\HW3\\functions.py'>

## 1: Upload data 
## 2: Create documents
First mission was to split and write our dataset into tab separated files for each line. To do this we:
 - create a dataframe with our data
 - removed new lines (\n)
 - transformed df in a list
 - create a tsv file with \t tab space for each row from the df1 list

In [17]:
#read csv as data frame with pandas
df = pd.read_csv("data/Airbnb_Texas_Rentals.csv")

#remove \\n new line from the description
df = df.replace(r'\\n', " ", regex = True)

#make the df as list
df1 = df.values.tolist()
#remove the index from df1

#create a tsv file with \t tab space for each row from the df1 list

for row in df1:
    with open("data/doc_tsv/doc_" +str(df1.index(row))+ ".tsv", 'w', newline='',encoding='utf8') as f_output:
        tsv_output = csv.writer(f_output, delimiter='\t',)
        tsv_output.writerow(row[1:]) #remove first row with names
        
for i in df1:
    i.remove(i[0])

# 3: Search Engine
#### All the data frame was preprocessed by

1) Removing stop words

2) Removing punctuation

3) Stemming

4) Tokenization

To make our computational steps faster we applied the preprocessing only to description and title columns

In [18]:
stopwords = stopwords.words('english') # get english stop words
#we tokenize except for the symbol $ that remains with the value
tokenizer = RegexpTokenizer("[\w'\$]+")
#stemming
st = PorterStemmer()

#tokenize description and title
df.description = df.description.apply(str).apply(tokenizer.tokenize)
df.title = df.title.apply(str).apply(tokenizer.tokenize)

#remove stop word and punctuation from description and title
df.description = df.description.apply(lambda x : [item for item in x if item not in stopwords])
df.title = df.title.apply(lambda x : [item for item in x if item not in stopwords])

#remove stemmings from descr and title
df.description = df.description.apply(lambda x : [st.stem(y) for y in x])
df.title = df.title.apply(lambda x : [st.stem(y) for y in x])


## 3.1 Conjunctive query

### 3.1.1 Create your index!

 To create an inverted index, we first split the content field of each document into separate words, create a sorted list of all the unique terms, and the list in which documents each term appears. Before building the index, we created a file named vocabulary, that maps each word to an integer (term_id). Then we created our inverted index.

We decide to store vocabulary, index and inverted index into JSON files to let them easily accessible.

In [19]:
#create the list of all the words
voc = []

for i in list(df.description):
    for j in i:
        voc.append(j)
for i in list(df.title):
    for j in i:
        voc.append(j)
voc = set(voc) #convert the list to a set of distinct words
num = list(voc) #make voc as list
vocabulary = {} #create empty dictionary

for i in num:
    vocabulary.update({num.index(i):i}) # get a key for each value of voc

#we save the vocabulary as json file
with open("vocabulary.json", "w", encoding = "utf8") as myfile:
    myfile.write(json.dumps(vocabulary))

It can be observed the presence of some words in oriental characters (logograms). We decide to not remove them because since we are representing real data, it can not be excluded that the query or the websites are in another language.


In [20]:
#create dict from df that assign for each rows (doc) all the words in
newdict = {}
for index, row in df.iterrows():
    newdict["doc_" + str(index)] = list(row["description"]) + list(row["title"])

In [21]:
with open("index.json", "w", encoding = "utf8") as myfile:#write normal index to json
    myfile.write(json.dumps(newdict))

In [22]:
#create inverted index called inverse

inverse = dict() 
for key in newdict: 
    # Go through the list that is saved in the dict:
    for item in set(newdict[key]):
        # Check if in the inverted dict the key exists
        if item not in inverse: 
            # If not create a new list
            inverse[item] = [key] 
        else: 
            inverse[item].append(key) 


In [23]:
# change the keys of inverse with the keys of vocabulary
for i in vocabulary:
    inverse[i] = inverse[vocabulary[i]]
    del inverse[vocabulary[i]]
#save inverted index as json
with open("inverted_index.json", "w", encoding = "utf8") as myfile:
    myfile.write(json.dumps(inverse))

### 3.1.2 Execute the query
##### What documents do we want?
The final output of the query its return the following information for each of the selected documents:

* Title
* Description
* City
* Clickable Url 

Non-existing queries its generate a warning message: "result for your query doesn't exist"

In [26]:
# Let's try "garden pergola":
functions.searchEngine_1()

Insert your query: pergola garden


,Title,Description,City,URL
0,"Home Away from Home in Spring, TX",Beautiful 2100 sq. ft house! Three bedroom and TV room. Roomy master bedroom with spacious bathroom. Bathroom has both a shower and garden tub! Back yard has a pergola and gazebo as well as patio furniture!,Spring,https://www.airbnb.com/rooms/1521227?location=Cleveland%2C%20TX


In [27]:
# As we have already said at the beginning, our query with no english words can produces a result.
# es 别墅二层独立房间 as input
functions.searchEngine_1()

Insert your query: 别墅二层独立房间


,Title,Description,City,URL
0,"别墅二层独立房间,二层共3个房间分别出租,二层独立客厅及卫生间","别墅位于安全安静的社区,二层为三室一厅一卫,三个房间分别出租,二层全订有额外优惠,床上用品及厨房用品齐全",McKinney,https://www.airbnb.com/rooms/16961346?location=Allen%2C%20TX
1,"别墅二层独立房间,二层共3个房间分别出租,二层独立客厅及卫生间","别墅位于安全安静的社区,二层为三室一厅一卫,三个房间分别出租,二层全订有额外优惠,床上用品及厨房用品齐全",McKinney,https://www.airbnb.com/rooms/16961346?location=Anna%2C%20TX


In [28]:
# check for a non-existent word as "abcdefg"
functions.searchEngine_1()

Insert your query: jdksdfjnrf
There are no results that match your query.


## 3.2 Conjunctive query & Ranking score

### 3.2.1 Inverted index


In [29]:
vocablist = [x for x in vocabulary.keys()] #list of words
corpus = json.loads(open('index.json').read())
inverted_index = json.loads(open('inverted_index.json').read())
vocab = json.loads(open('vocabulary.json').read())


What we need now is to calculate che IDF and TF - IDF, according to the formulas: 
- $TF = \frac{N_{(x,y)}}{N_{(*,y)}}$
- $IDF = log[1 + (\frac{D}{D_x})]$ <fr>

Where:
- $N_{(x,y)}$ is the number of times that the word $X$ is in the document $D_y$;
- $N_{(*,y)}$ is the total number of the words in the document;
- $D$ is the total number of documents;
- $D_x$ is the number of documents in which the word $X$ appears at least once.

In [30]:
# calculate the IDF
idf = {}
for term in inverted_index:
    term1 = vocab[term]
    idf[term] = math.log(1+(len(corpus)/len(term1)))
with open("idf_index.json", "w", encoding = "utf8") as myfile:
    myfile.write(json.dumps(inverted_index))

In [31]:
#calculate the TF - IDF
for term in inverted_index:
    term1 = vocab[term]
    for doc in inverted_index[term]:
        count = (corpus[doc].count(term1)/len(corpus[doc]))*idf[term]
        inverted_index[term][inverted_index[term].index(doc)]=(doc,count)
        
with open("tfidf_index.json", "w", encoding = "utf8") as myfile:
    myfile.write(json.dumps(inverted_index))
    

### 3.2.2 Execute the query - Cosine Similarity


In [33]:
functions.searchEngine_2()

Insert your query: dog garden 
How many results do you want to see? 5


,Title,Description,City,URL,Similarity
0,Ravenshire-LLC Retreat -- Mi Casa es tu Casa,"Perfect for ACU events: WALK to Summit, Homecoming, Sing Song, and Graduation! Close to Hendrick Hospital & Dyess. Perfect for Professionals, Traveling Nurses, Writers, & their FAMILIES. ALL amenities are provided: Housekeeper, Gardener, Property Manager. Complete turn-key living. Good place for kids and pets (dogs). See Ravenshire-LLC for more information.",Abilene,https://www.airbnb.com/rooms/15420018?location=Cisco%2C%20TX,0.444241
1,Pool Cabana in Historic Country Setting,"This cozy cottage sits on a beautiful acre nestled under a canopy of large hardwoods. The property backs up to the city park, sporting a walking trail, tennis court, dog park, playgrounds, and the meandering 10-mile Creek. Staying in Historic Lancaster is like stepping back into tiime. The shady sidewalk takes you to the town square, train depot and rose garden. Nature preserves, groceries, and restaurants are 5 minutes away. It is 25 minutes from Dallas and 10 minutes from I 35, I 20, & I 45.",Lancaster,https://www.airbnb.com/rooms/16644591?location=Corsicana%2C%20TX,0.444241
2,Quiet Study with Private Entrance,"This personal study sits on a beautiful acre nestled under a canopy of large hardwoods. The property backs up to the city park, sporting a walking trail, tennis court, dog park, playgrounds, and the meandering 10-mile Creek. Staying in Historic Lancaster is like stepping back into tiime. The shady sidewalk takes you to the town square, train depot and rose garden. Nature preserves, groceries, and restaurants are 5 minutes away. It is 25 minutes from Dallas & 10 minutes from I 35, I 20, & I 45.",Lancaster,https://www.airbnb.com/rooms/16646198?location=Corsicana%2C%20TX,0.444241
3,Ravenshire-LLC Retreat -- Mi Casa es tu Casa,"Perfect for ACU events: WALK to Summit, Homecoming, Sing Song, and Graduation! Close to Hendrick Hospital & Dyess. Perfect for Professionals, Traveling Nurses, Writers, & their FAMILIES. ALL amenities are provided: Housekeeper, Gardener, Property Manager. Complete turn-key living. Good place for kids and pets (dogs). See Ravenshire-LLC for more information.",Abilene,https://www.airbnb.com/rooms/15420018?location=Abilene%2C%20TX,0.444241
4,Maison d'Etre,"Perfect ACL, F1, SXSW location! A colorful strawbale constructed home minutes from downtown Austin is nestled on two acres with chickens, ornamental pond, rooftop terrace, porches, parking, gardens, a big dog, and honeybees. Come In, We're Awesome!",Austin,https://www.airbnb.com/rooms/5037508?location=Bee%20Cave%2C%20TX,0.444241


# 4. Define a new score!

In [36]:
df["price"] = df["average_rate_per_night"].replace({'\$':''}, regex = True)
df["price"] = df["price"].replace(np.nan, '0', regex=True)

df.price = df.price.astype("int") #make the price as int


df["bedroom"] = df["bedrooms_count"].replace({'Studio':'1'}, regex=True)
df["bedroom"] = df["bedroom"].replace(np.nan, '0', regex=True)
df.bedroom = df.bedroom.astype("int")

df["price_score"] = abs((df["price"] - df["price"].max() )/(df["price"].max() - df["price"].min()) )
df["bed_score"] = (df["bedroom"] - df["bedroom"].min() )/(df["bedroom"].max() - df["bedroom"].min())

df["av_score"] = (df["price_score"] + df["bed_score"])/2

av_price_city = df.groupby("city")["price"].agg(['mean','count','max','min'])

av_price_city["min_int"] = (av_price_city["mean"] - av_price_city["min"])/3
av_price_city["max_int"] = (av_price_city["max"] - av_price_city["mean"])/3

newdf = pd.merge(df, av_price_city, on="city")

newdf['city_score'] = newdf.apply(functions.city_score, axis=1)

newdf["city_score"] = (newdf["city_score"] - newdf["city_score"].min() )/(newdf["city_score"].max() - newdf["city_score"].min())

newdf.loc[newdf['count'] < 5, 'city_score'] = 0.5 

newdf=newdf.drop(['mean','count','max','min','min_int','max_int', "av_score"],axis=1)

newdf.head()

,Unnamed: 0,average_rate_per_night,bedrooms_count,city,date_of_listing,description,latitude,longitude,title,url,price,bedroom,price_score,bed_score,city_score
0,1,$27,2,Humble,May 2016,"[welcom, stay, privat, room, queen, bed, detac...",30.020138,-95.293996,"[2, privat, room, bathroom, 10min, iah, airport]",https://www.airbnb.com/rooms/18520444?location...,27,2,0.9973,0.153846,1.00
1,54,$100,2,Humble,January 2016,"[My, home, built, 2016, 5, min, airport, quiet...",30.019380,-95.308188,"[cozi, second, floor, minut, iah]",https://www.airbnb.com/rooms/16925131?location...,100,2,0.9900,0.153846,1.00
2,56,$81,3,Humble,January 2017,"[entir, 3, bedroom, 2, full, bath, home, quiet...",30.000943,-95.258168,"[entir, home, 5, mile, from, iah, block, hwi, ...",https://www.airbnb.com/rooms/17006337?location...,81,3,0.9919,0.230769,1.00
3,92,$200,4,Humble,November 2014,"[4, bedroom, 1, live, room, 1, studi, room, 3,...",29.929538,-95.250962,"[2, stori, home, bush, int'l, airport]",https://www.airbnb.com/rooms/4520039?location=...,200,4,0.9800,0.307692,0.75
4,135,$175,3,Humble,February 2016,"[3, bedroom, 2, 5, bathroom, custom, builder, ...",29.986569,-95.228184,"[singl, stori, golf, cours, home, 15, minut, iah]",https://www.airbnb.com/rooms/16496167?location...,175,3,0.9825,0.230769,0.75


In [37]:
newdf["new_score"] = (newdf["price_score"] + newdf["bed_score"] + newdf["city_score"])/3

newdf.head()

scores = pd.DataFrame(columns = ["doc", "av_score"])

scores["doc"] = newdf.index

scores["av_score"]=newdf.new_score



newlist=scores.values.tolist()

scores_dict = {}

for i in newlist:
    scores_dict["doc_"+str(int(i[0]))]=i[1]
with open("scores_dict.json", "w", encoding = "utf8") as myfile:
    myfile.write(json.dumps(scores_dict))

In [38]:
functions.searchEngine_3()

Insert your query: pergola
please enter the filters you want separated by space or leave blank:
type
1 for city
2 for bedrooms
3 for price
1 2
Type the name of a city: San Antonio
Type the minimun and maximum number of bedroom separeted by space: 2 8


,Ranking,Title,Description,City,URL
1,1,"2,850 SF Cozy Home with a Private Pool","2,850 home on a quarter acre with solar heated pool. The home has 5 plus a loft with King size bed. There is a total of 7 beds and one sofa sleeper, so everyone can have their own bed and sleep well. The backyard has extensive landscaping and a huge gorgeous oak tree. The back patio is covered by a pergola covered in vines keeping it 15 degrees cooler. The backyard has a 6 foot fence and is ideal for pets. There is a dog park nearby.",San Antonio,https://www.airbnb.com/rooms/18774498?location=Castroville%2C%20TX
2,2,Modern Comfort and Entertainment,"Location is key with this stunning 4 bed / 2.5 bath well-maintained home located in the highly desirable neighborhood of Bulverde Village. Like to entertain when in S.A.? There is a huge patio with special yard lighting, pergola and bbq pit. Location is five minutes away from The Village Shopping Center at Stone Oak with an Alamo Drafthouse Cinema, multiple shops, and restaurants near by. Additionally, Walmart, Costco, HEB, a few malls and the international airport only 15 minutes away.",San Antonio,https://www.airbnb.com/rooms/19114219?location=Cibolo%2C%20TX
3,3,Modern Comfort and Entertainment,"Location is key with this stunning 4 bed / 2.5 bath well-maintained home located in the highly desirable neighborhood of Bulverde Village. Like to entertain when in S.A.? There is a huge patio with special yard lighting, pergola and bbq pit. Location is five minutes away from The Village Shopping Center at Stone Oak with an Alamo Drafthouse Cinema, multiple shops, and restaurants near by. Additionally, Walmart, Costco, HEB, a few malls and the international airport only 15 minutes away.",San Antonio,https://www.airbnb.com/rooms/19114219?location=Bulverde%2C%20TX


##  FOR BONUS STEP LOOK AT MAPS.IPYNB